In [1]:
import pandas as pd
import numpy as np

from pymcdm.methods import PROMETHEE_II
from collections import defaultdict

In [2]:
df = pd.read_csv("paragrapghs_with_topisc.csv.zip", index_col=0)
df.head()

/tmp/ipykernel_37662/3613625376.py:1: DtypeWarning: Columns (56,59,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragrapghs_with_topisc.csv.zip", index_col=0)


,Unnamed: 0,rsspp_index,index,company,year,sector,report_type,paragraph,original_text,cleaned_text,...,!Кибербезопасность,max_topic_cos,max_score_cos,count_words,second_max_topic_cos,second_max_score_cos,max_topic_exp,max_score_exp,second_max_topic_exp,second_max_score_exp
0,0.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,0.0,та новая выс,тот новый выс,...,0.0,Отчетность и прозрачность,0.190071,1.0,NaN,NaN,Отношения с инвесторами,-4170.0,Отчетность и прозрачность,-4784.0
1,1.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,1.0,взаимодействия,взаимодействие,...,0.0,Корпоративное управление,0.425628,1.0,Лидерство,0.292690,Потребители. Сервис и коммуника,-11032.0,Потребители. Удовлетворенность,-11174.0
2,2.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,2.0,годовой отчет за 2020 год,годовой отчёт 2020 год,...,0.0,Корпоративное управление,0.425628,1.0,Лидерство,0.289862,Потребители. Сервис и коммуника,-8674.0,Потребители. Удовлетворенность,-8734.0
3,3.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,3.0,содержание,содержание,...,0.0,Отчетность и прозрачность,0.190071,1.0,NaN,NaN,Отношения с инвесторами,-3750.0,Отчетность и прозрачность,-3945.0
4,4.0,674.0,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,4.0,2,2,...,0.0,Обучение и развитие,0.214768,1.0,Устойчивое развитие,0.176117,Социальные инвестиции и благотв,-13625.0,Обучение и развитие,-13907.0


In [4]:
df.columns

Index(['Unnamed: 0', 'rsspp_index', 'index', 'company', 'year', 'sector',
       'report_type', 'paragraph', 'original_text', 'cleaned_text',
       'Экология в целом', 'Климат', 'Энергия', 'Воздух', 'Вода',
       'Отходы и циклическая экономика', 'Биоразнообразие',
       'Рекультивация земель', 'Экологичность продукта', '!Персонал в целом',
       'Обучение и развитие', 'Сотрудники. Вовлеченность и мот',
       'Оплата труда', 'Сотрудники. Здоровье и благопол',
       'Сотрудники. Наем и увольнение', 'Сотрудники. Корпоративная культ',
       'Сотрудники. Безопасность и охра', 'Сотрудники. Профсоюз и Коллекти',
       'Потребители. Доступность', 'Потребители. Сервис и коммуника',
       'Потребители. Персональные данны', 'Потребители. Здоровье и благопо',
       'Потребители. Маркетинг, продажи', 'Потребители. Удовлетворенность',
       'Потребители. Качество и безопас', 'Потребители. Ценовая политика',
       'Поставщики в целом', 'Малый и локальный бизнес',
       'Поставщики. Рабо

In [15]:
gr = (
    df.drop(
        columns=[
            "Unnamed: 0",
            "rsspp_index",
            "index",
            "sector",
            "year",
            "report_type",
            "paragraph",
            "original_text",
            "cleaned_text",
            "max_topic_cos",
            "max_score_cos",
            "count_words",
            "second_max_topic_cos",
            "second_max_score_cos",
            "max_topic_exp",
            "max_score_exp",
            "second_max_topic_exp",
            "second_max_score_exp",
        ]
    )
    .groupby("company")
    .mean()
    .dropna(how="all")
)
gr.head()

,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,Экологичность продукта,!Персонал в целом,...,Отчетность и прозрачность,Отношения с инвесторами,!Инновации,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,!Кибербезопасность
company,,,,,,,,,,,,,,,,,,,,,
reports/GV Gold,0.007228,0.024504,0.023391,0.011764,0.019744,0.013694,0.006476,0.011152,0.010913,0.0,...,0.024308,0.001417,0.0,0.0,0.020799,0.021285,0.005739,0.012695,0.026362,0.0
reports/«МЕТАЛЛОИНВЕСТ»,0.005318,0.017873,0.010172,0.007300,0.014439,0.015223,0.002570,0.003172,0.016063,0.0,...,0.026322,0.002332,0.0,0.0,0.031231,0.014672,0.004013,0.016205,0.025135,0.0
reports/АО «СУЭК»,0.011790,0.012718,0.009848,0.011270,0.015054,0.016218,0.003856,0.002934,0.011745,0.0,...,0.038851,0.001876,0.0,0.0,0.034406,0.017499,0.001861,0.023446,0.027961,0.0
reports/Газпромбанк,0.005860,0.024217,0.012030,0.009286,0.008939,0.005113,0.003825,0.002933,0.006976,0.0,...,0.032912,0.001359,0.0,0.0,0.026048,0.016484,0.006207,0.005743,0.035527,0.0
reports/Группа ЛСР,0.007281,0.025014,0.007616,0.013441,0.015713,0.011888,0.006987,0.006941,0.008769,0.0,...,0.034395,0.000992,0.0,0.0,0.026231,0.027838,0.002592,0.008336,0.023474,0.0


In [17]:
matrix = gr.to_numpy()
matrix.shape

(20, 45)

In [19]:
weights = np.array([1 / 45] * matrix.shape[1])
types = np.ones(matrix.shape[1], dtype=int)

In [21]:
body = PROMETHEE_II('usual')

In [24]:
[round(preference, 2) for preference in body(matrix, weights, types)]

[0.06,
 0.06,
 0.18,
 0.15,
 0.07,
 -0.16,
 -0.27,
 0.02,
 -0.01,
 -0.02,
 0.18,
 0.06,
 -0.75,
 0.13,
 0.17,
 0.11,
 0.22,
 0.05,
 -0.27,
 0.02]